<a href="https://colab.research.google.com/github/machiwao/pjdsc-optima-prime-sakayan-express/blob/main/SakayanExpress_DemandMapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sakayan Express - Demand Mapping
Map passenger demand on a respective location. Predict number of passengers at a given time

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Dataset Loading

In [ ]:
df = pd.read_csv("/content/0711_BELGA_withFuelData.csv")

In [ ]:
df.head()

,id,driver,datePH,longitude,latitude,altitude,gpsSpeed,timeStamp,calEngineLoad,vehSpeed,...,Altitude,Accuracy,Hour,Min,Sec,Feedno,Withpass,Withevent,Cumpass,Pass
0,64ac640d8311b05aa142074b,Roly Belga,20230711,120.989040,14.507968,-18,1,2023-07-10T20:00:55.000+0000,22,0,...,50.5,3.90,4.0,0.0,55.0,532.0,1.0,2.0,14.0,0.0
1,64ac640d8311b05aa142074a,Roly Belga,20230711,120.988890,14.507952,-34,0,2023-07-10T20:00:56.000+0000,22,0,...,50.5,3.90,4.0,0.0,56.0,533.0,1.0,2.0,14.0,0.0
2,64ac640b8311b05aa1420746,Roly Belga,20230711,120.988825,14.507938,-40,1,2023-07-10T20:00:57.000+0000,22,0,...,50.5,3.90,4.0,0.0,57.0,534.0,1.0,2.0,14.0,0.0
3,64ac640b8311b05aa1420745,Roly Belga,20230711,120.988810,14.507935,-39,0,2023-07-10T20:00:58.000+0000,22,0,...,50.5,3.42,4.0,0.0,58.0,535.0,1.0,2.0,14.0,0.0
4,64ac640b8311b05aa1420744,Roly Belga,20230711,120.988802,14.507937,-38,0,2023-07-10T20:00:59.000+0000,22,0,...,50.5,3.42,4.0,0.0,59.0,536.0,1.0,2.0,14.0,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59962 entries, 0 to 59961
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   59962 non-null  object 
 1   driver               59962 non-null  object 
 2   datePH               59962 non-null  int64  
 3   longitude            59962 non-null  float64
 4   latitude             59962 non-null  float64
 5   altitude             59962 non-null  int64  
 6   gpsSpeed             59962 non-null  int64  
 7   timeStamp            59962 non-null  object 
 8   calEngineLoad        59962 non-null  int64  
 9   vehSpeed             59962 non-null  int64  
 10  throttlePosition     59962 non-null  int64  
 11  ambientAirTemp       59962 non-null  int64  
 12  engineOilTemp        59962 non-null  int64  
 13  Speed                59962 non-null  int64  
 14  fuelPressure         59962 non-null  int64  
 15  engineRPM            59962 non-null 

In [ ]:
df.describe()

,datePH,longitude,latitude,altitude,gpsSpeed,calEngineLoad,vehSpeed,throttlePosition,ambientAirTemp,engineOilTemp,...,Altitude,Accuracy,Hour,Min,Sec,Feedno,Withpass,Withevent,Cumpass,Pass
count,59962.0,59962.000000,59962.000000,59962.000000,59962.000000,59962.000000,59962.000000,59962.000000,59962.000000,59962.000000,...,53385.000000,53385.000000,53385.000000,53385.000000,53385.000000,53385.000000,53385.000000,53385.000000,53385.000000,53385.000000
mean,20230711.0,121.015852,14.582620,31.943781,12.439695,40.623378,13.531353,16.221574,31.234882,91.116074,...,68.544674,7.283566,11.684256,29.035984,29.508439,27089.408954,1.032799,1.976042,38.653255,0.000468
std,0.0,0.024735,0.054249,18.870216,16.248403,22.730954,16.755636,26.054464,2.799451,2.627240,...,18.453354,8.305447,4.872378,16.987965,17.326651,15347.426785,0.178113,0.152920,16.971604,0.154785
min,20230711.0,120.983452,14.507295,-157.000000,0.000000,0.000000,0.000000,0.000000,27.000000,86.000000,...,-168.810000,2.150000,4.000000,0.000000,0.000000,532.000000,1.000000,1.000000,7.000000,-1.000000
25%,20230711.0,120.991318,14.537463,18.000000,0.000000,31.000000,0.000000,0.000000,29.000000,89.000000,...,55.800000,3.900000,8.000000,15.000000,15.000000,13775.000000,1.000000,2.000000,26.000000,0.000000
50%,20230711.0,121.009363,14.564530,31.000000,0.000000,41.000000,4.000000,0.000000,31.000000,91.000000,...,65.500000,4.250000,12.000000,29.000000,30.000000,27057.000000,1.000000,2.000000,35.000000,0.000000
75%,20230711.0,121.039225,14.644352,46.000000,27.000000,51.000000,28.000000,30.000000,34.000000,93.000000,...,81.000000,9.030000,16.000000,44.000000,45.000000,40396.000000,1.000000,2.000000,53.000000,0.000000
max,20230711.0,121.059888,14.657928,120.000000,62.000000,100.000000,65.000000,100.000000,39.000000,100.000000,...,309.660000,335.900000,20.000000,59.000000,59.000000,53641.000000,2.000000,2.000000,85.000000,1.000000


In [ ]:
df.isnull().sum()

,0
id,0
driver,0
datePH,0
longitude,0
latitude,0
altitude,0
gpsSpeed,0
timeStamp,0
calEngineLoad,0
vehSpeed,0


In [ ]:
df.duplicated().sum()

0